In [1]:
class BinaryTree:
    def __init__(self, key, leftTree=None, rightTree=None):
        self.key = key
        self.leftTree = leftTree
        self.rightTree = rightTree

    def setKey(self, key):
        self.key = key

    def getKey(self):
        return self.key

    def getLeftTree(self):
        return self.leftTree

    def getRightTree(self):
        return self.rightTree

    def insertLeft(self, key):
        if self.leftTree is None:
            self.leftTree = BinaryTree(key)
        else:
            new_node = BinaryTree(key, leftTree=self.leftTree)
            self.leftTree = new_node

    def insertRight(self, key):
        if self.rightTree is None:
            self.rightTree = BinaryTree(key)
        else:
            new_node = BinaryTree(key, rightTree=self.rightTree)
            self.rightTree = new_node

    def printPreorder(self, level=0):
        print(str(level * '-') + str(self.key))
        if self.leftTree is not None:
            self.leftTree.printPreorder(level + 1)
        if self.rightTree is not None:
            self.rightTree.printPreorder(level + 1)
            
    def printInorder(self, level=0):
        if self.leftTree is not None:
            self.leftTree.printInorder(level + 1)
        print(str(level * '-') + str(self.key))
        if self.rightTree is not None:
            self.rightTree.printInorder(level + 1)

    def printPostorder(self, level=0):
        if self.leftTree is not None:
            self.leftTree.printPostorder(level + 1)
        if self.rightTree is not None:
            self.rightTree.printPostorder(level + 1)
        print(str(level * '-') + str(self.key))


class Stack:
    def __init__(self):
        self.items = []

    def isEmpty(self):
        return self.items == []

    def push(self, item):
        self.items.append(item)

    def pop(self):
        if not self.isEmpty():
            return self.items.pop()
        else:
            raise IndexError("Pop from an empty stack")

    def peek(self):
        if not self.isEmpty():
            return self.items[-1]
        else:
            raise IndexError("Peek from an empty stack")

    def size(self):
        return len(self.items)


class Node:
    def __init__(self, key, item):
        self.key = key
        self.item = item
        self.next = None

class LinkedList:
    def __init__(self):
        self.head = None

    def append(self, key, item):
        new_node = Node(key, item)
        if self.head is None:
            self.head = new_node
        else:
            current = self.head
            while current.next:
                current = current.next
            current.next = new_node

    def find(self, key):
        current = self.head
        while current:
            if current.key == key:
                return current.item
            current = current.next
        return None

    def sort(self):
        if self.head is None or self.head.next is None:
            return

        size = 1
        current = self.head
        while current.next:
            size += 1
            current = current.next

        step = 1
        while step < size:
            self.head = self.merge_pass(self.head, step)
            step *= 2

    def merge_pass(self, head, step):
        current = head
        prev_tail = None
        new_head = None

        while current:
            left = current
            right = self.split(left, step)
            current = self.split(right, step)

            merged = self.sorted_merge(left, right)

            if prev_tail:
                prev_tail.next = merged
            else:
                new_head = merged

            while prev_tail and prev_tail.next:
                prev_tail = prev_tail.next

        return new_head

    def split(self, head, step):
        if head is None:
            return None
        for _ in range(step - 1):
            if head.next is None:
                break
            head = head.next
        next_head = head.next
        head.next = None
        return next_head

    def sorted_merge(self, a, b):
        if a is None:
            return b
        if b is None:
            return a

        if a.key <= b.key:
            result = a
            result.next = self.sorted_merge(a.next, b)
        else:
            result = b
            result.next = self.sorted_merge(a, b.next)
        return result

In [2]:
class ParseTree:
    def __init__(self):
        self.root = None
        self.variables = {}

    def tokenise(self, exp):
        raw = list(exp.replace(" ", ""))
        tokens = []
        i = 0

        while i < len(raw):
            if raw[i] == '*' and i + 1 < len(raw) and raw[i + 1] == '*':
                tokens.append('**')
                i += 2
            elif raw[i].isdigit() or raw[i] == '-':
                num = ''
                while i < len(raw) and (raw[i].isdigit() or raw[i] == '-'):
                    num += raw[i]
                    i += 1
                tokens.append(num)
            else:
                tokens.append(raw[i])
                i += 1
        return tokens


    def infix_to_postfix(self, expression):
        precedence = {'+': 1, '-': 1, '*': 2, '/': 2, '**': 3, '(': 0}
        stack = Stack()
        postfix = []
        tokens = self.tokenise(expression)

        def is_operand(t):
            return t.isnumeric() or t in self.variables or (t[0] == '-' and t[1:].isnumeric())

        for i, token in enumerate(tokens):
            if is_operand(token):
                postfix.append(token)

                if i < len(tokens) - 1 and tokens[i + 1] == '(':
                    while not stack.isEmpty() and precedence[stack.peek()] >= precedence['*']:
                        postfix.append(stack.pop())
                    stack.push('*')

            elif token == '(':
                if i > 0 and (is_operand(tokens[i - 1]) or tokens[i - 1] == ')'):
                    while not stack.isEmpty() and precedence[stack.peek()] >= precedence['*']:
                        postfix.append(stack.pop())
                    stack.push('*')
                stack.push(token)

            elif token == '-':
                if i == 0 or tokens[i - 1] in '+-*/(**':
                    postfix.append('0')
                while not stack.isEmpty() and precedence[stack.peek()] >= precedence[token]:
                    postfix.append(stack.pop())
                stack.push(token)

            elif token in precedence:
                while not stack.isEmpty() and precedence[stack.peek()] >= precedence[token]:
                    postfix.append(stack.pop())
                stack.push(token)

            elif token == ')':
                topToken = stack.pop()
                while topToken != '(':
                    postfix.append(topToken)
                    topToken = stack.pop()

        while not stack.isEmpty():
            postfix.append(stack.pop())

        return ' '.join(postfix)

    def build_tree(self, postfix_expr):
        token_list = self.tokenise(postfix_expr)
        stack = Stack()

        for token in token_list:
            print("Current Token:", token)  # Diagnostic print
            if token in "+-*/**":
                if stack.size() < 2:
                    raise ValueError(f"Invalid postfix expression: Not enough operands for operator {token}")

                node = BinaryTree(token)
                node.rightTree = stack.pop()
                node.leftTree = stack.pop()
                stack.push(node)
            else:
                if token in self.variables:
                    token = self.variables[token]
                stack.push(BinaryTree(token))
        print("Final Stack Size:", stack.size())  # Diagnostic print
        if stack.size() != 1:
            raise ValueError("Invalid expression")

        self.root = stack.pop()

    def evaluate_tree(self, node):
        if node is None:
            return None
        if node.leftTree is None and node.rightTree is None:
            try:
                return float(node.key)
            except ValueError:
                return None
        left_val = self.evaluate_tree(node.leftTree)
        right_val = self.evaluate_tree(node.rightTree)

        if node.key == '+':
            return left_val + right_val
        elif node.key == '-':
            return left_val - right_val
        elif node.key == '*':
            return left_val * right_val
        elif node.key == '/':
            return left_val / right_val
        elif node.key == '**':
            return left_val ** right_val

    def evaluate_expression(self, expression):
        tokens = self.tokenise(expression)
        for i, token in enumerate(tokens):
            if token in self.variables:
                tokens[i] = str(self.variables[token])

        postfix = self.infix_to_postfix(' '.join(tokens))
        print("Postfix Expression:", postfix)  # Diagnostic print
        self.build_tree(postfix)

        return self.evaluate_tree(self.root)

In [3]:
expression_list = LinkedList()
expression_list.append('a', '5')
expression_list.append('b', 'a * 5')
expression_list.append('c', '-a(5)')
expression_list.append('d', 'x')

parse_tree = ParseTree()

# Evaluate and print expressions
current = expression_list.head
while current:
    result = parse_tree.evaluate_expression(current.item)
    if result is not None:
        parse_tree.variables[current.key] = result
    print(f'{current.key} = {current.item.replace(" ", "")} => {result}')
    current = current.next


Postfix Expression: 5
Current Token: 5
Final Stack Size: 1
a = 5 => 5.0
Postfix Expression: 5 0 5 *
Current Token: 505
Current Token: *


ValueError: Invalid postfix expression: Not enough operands for operator *